In [1]:
%load_ext autoreload
%autoreload 2

%load_ext watermark
%watermark --iversions

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import io
from utils.benchmark import Bench
from tqdm.notebook import tqdm

from utils.dgps import generate_benchmark_data
from utils.estimators import (
    twfe_fixest, 
    twfe_fixest_compressed, 
    twfe_statsmodels, 
    duck_mundlak, 
    event_study_fixest, 
    duck_mundlak_event, 
    event_study_statsmodels
)
from utils.benchmark import Bench
from itertools import product


In [2]:
N_values = [10_000, 100_000]
T_values = [14, 28, 42]
models = [
    twfe_statsmodels, 
    twfe_fixest, 
    twfe_fixest_compressed, 
    event_study_statsmodels, 
    event_study_fixest, 
    duck_mundlak, 
    duck_mundlak_event
]
T0_values = [7]
iter = 3

all_benchmarks_df = pd.DataFrame()

# Initialize the tqdm progress bar for the outer loop
with tqdm(total= len(models) * len(N_values) * len(T_values) * len(T0_values), desc="Processing benchmarks") as pbar:

    for model, N, T, T0 in product(models, N_values, T_values, T0_values):

        print("Model:", model)
        bench = Bench(N=N, T=T, T0=T0, iter=iter)

        # Suppress tqdm output in called functions for Jupyter
        temp_stdout = io.StringIO()
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = temp_stdout
        sys.stderr = temp_stdout
        try:
            bench.mark(fun=model)
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

        # Update the progress bar manually
        pbar.update(1)

        # Collect results
        bench_df = bench.to_dataframe()
        bench_df["N"] = N
        bench_df["T"] = T
        bench_df["T0"] = T0

        bench_df.to_csv(f"data/benchmark_{model}_{N}_{T}_{T0}.csv")

        all_benchmarks_df = pd.concat(
            [all_benchmarks_df, bench_df], 
            axis=0
        )




Processing benchmarks:   0%|          | 0/6 [00:00<?, ?it/s]

Data loaded into DuckDB database: benchmarks.db
Data loaded into DuckDB database: benchmarks.db


## Load Data

In [ ]:
import pandas as pd
import glob
import os

all_files = glob.glob(os.path.join("data", "*.csv"))
all_benchmarks_list = [pd.read_csv(file) for file in all_files]

all_benchmarks_df = pd.concat(all_benchmarks_list, axis = 0)

models = [
    "duck_mundlak", 
    "twfe_fixest", 
    "twfe_statsmodels", 
    "twfe_fixest_compressed",
    "event_study_fixest", 
    "duck_mundlak_event", 
    "event_study_statsmodels"
]

In [ ]:
all_benchmarks_df_melted = all_benchmarks_df.melt(
    id_vars=['N', 'T', 'T0'],
    value_vars=models, 
    var_name='model',
    value_name='value'
)

median_run = (
    all_benchmarks_df_melted
        .groupby(["N", "T", "T0", "model"])
        .median()
        .reset_index()
)

In [ ]:
median_run_pooled = median_run[median_run.model.isin([
    "duck_mundlak", 
    "twfe_fixest", 
    "twfe_statsmodels", 
    "twfe_fixest_compressed"
])]

median_run_event = median_run[median_run.model.isin([
    "event_study_fixest", 
    "duck_mundlak_event", 
    "event_study_statsmodels"
])]


### Visualize Pooled Effect

In [ ]:
g = sns.FacetGrid(median_run_pooled, col="T", row="N", margin_titles=True, height=4, aspect=1)
g.map_dataframe(sns.barplot, x='model', y='value', hue='model', palette='Set2', legend=False)

# Adjust the x-axis labels' rotation for each subplot
for ax in g.axes.flat:
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

# Adjust layout
plt.yscale('log')
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Benchmarks of Panel Implementations')

plt.show()


## Visualize Event Study

In [ ]:
g = sns.FacetGrid(median_run_event, col="T", row="N", margin_titles=True, height=4, aspect=1)
g.map_dataframe(sns.barplot, x='model', y='value', hue='model', palette='Set2', legend=False)

# Adjust the x-axis labels' rotation for each subplot
for ax in g.axes.flat:
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

# Adjust layout
plt.yscale('log')
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Benchmarks of Panel Implementations')

plt.show()